# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:

output_file = "output_data/cities.csv"
# output_data_file

weather_df = pd.read_csv(output_file)

weather_data_df = weather_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,vigia,90,BR,2020-04-18 01:58:13+00:00,94,77.00,-0.86,-48.14,1.12
1,punta arenas,14,CL,2020-04-18 01:55:10+00:00,66,48.20,-53.15,-70.92,11.41
2,kodiak,90,US,2020-04-18 01:58:13+00:00,76,51.80,57.79,-152.41,10.29
3,chumikan,97,RU,2020-04-18 01:58:13+00:00,85,29.53,54.72,135.31,5.61
4,rikitea,1,PF,2020-04-18 01:54:40+00:00,63,77.36,-23.12,-134.97,1.99
...,...,...,...,...,...,...,...,...,...
561,chulym,2,RU,2020-04-18 01:59:35+00:00,79,45.75,55.10,80.96,8.68
562,palu,59,ID,2020-04-18 01:56:40+00:00,53,90.88,-0.89,119.87,2.62
563,asau,99,RO,2020-04-18 01:59:35+00:00,71,50.00,46.43,26.40,10.29
564,tarrafal,20,CV,2020-04-18 01:59:35+00:00,68,71.60,15.28,-23.77,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:

gmaps.configure(api_key=g_key)

In [13]:

humidity = weather_data_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_data_df[["Lat", "Lng"]]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
new_fit_df = weather_data_df.loc[(weather_data_df["Temp"] > 70) & (weather_data_df["Temp"] < 80) & (weather_data_df["Cloudiness"] == 0) & (weather_data_df["Wind Speed"]< 10), :]
new_fit_df = new_fit_df.dropna(how='any')
new_fit_df.reset_index(inplace=True)
del new_fit_df['index']
new_fit_df.head()

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,la romana,0,DO,2020-04-18 01:58:20+00:00,85,73.80,18.43,-68.97,9.42
1,moa,0,CU,2020-04-18 01:58:48+00:00,82,74.55,20.66,-74.94,8.41
2,san juan,0,AR,2020-04-18 01:55:14+00:00,34,71.82,-31.54,-68.54,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:

hotellist = []

for i in range(len(new_fit_df)):
    lat = new_fit_df.loc[i]['Lat']
    lng = new_fit_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
new_fit_df["Hotel Name"] = hotellist
new_fit_df = new_fit_df.dropna(how='any')
new_fit_df.head()

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,la romana,0,DO,2020-04-18 01:58:20+00:00,85,73.80,18.43,-68.97,9.42,La Romana
1,moa,0,CU,2020-04-18 01:58:48+00:00,82,74.55,20.66,-74.94,8.41,Moa
2,san juan,0,AR,2020-04-18 01:55:14+00:00,34,71.82,-31.54,-68.54,6.91,San Juan


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_fit_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))